In [1]:
#https://github.com/backstopmedia/tensorflowbook/blob/master/chapters/04_machine_learning_basics/softmax.py
import os
import time
import tensorflow as tf
import numpy as np

In [2]:
W = tf.Variable(tf.zeros([4,3]),name='weight')
b = tf.Variable(tf.zeros([3]),name="bias")

In [3]:
def read_csv(batch_size, file_name, record_defaults):
    filename_queue = tf.train.string_input_producer([os.path.join(os.getcwd(), file_name)]) 

    reader = tf.TextLineReader()
    key, value = reader.read(filename_queue)

    # decode_csv will convert a Tensor from type string (the text line) in
    # a tuple of tensor columns with the specified defaults, which also
    # sets the data type for each column
    decoded = tf.decode_csv(value, record_defaults=record_defaults)

    # batch actually reads the file and loads "batch_size" rows in a single tensor
    return tf.train.shuffle_batch(decoded,
                                  batch_size=batch_size,
                                  capacity=batch_size * 50,
                                  min_after_dequeue=batch_size)


In [4]:
def inputs():
    """"""
    # https://archive.ics.uci.edu/ml/datasets/Iris
    # 原始数据最后2行为空，需要删除掉空行
    sepal_length, sepal_width, petal_length, petal_width, label =\
        read_csv(100, "./data/iris/iris.data", [[0.0], [0.0], [0.0], [0.0], [""]])

    # convert class names to a 0 based class index.
    # argmax ？
    label_number = tf.to_int32(tf.argmax(tf.to_int32(tf.stack([
        tf.equal(label, ["Iris-setosa"]),
        tf.equal(label, ["Iris-versicolor"]),
        tf.equal(label, ["Iris-virginica"])
    ])), 0))

    # stack all the features that we care about in a single matrix;
    # We then transpose to have a matrix with one example per row and one feature per column.
    features = tf.transpose(tf.stack([sepal_length, sepal_width, petal_length, petal_width]))

    return features, label_number


In [5]:
def combine_inputs(X): 
    return tf.matmul(X,W) + b #X,W不能换位置

def inference(X):
    """推断模型"""
    return tf.nn.softmax(combine_inputs(X)) 

def loss(X,Y):
    """损失函数，训练数据X，期望输出Y，计算损失"""
    # sparse_softmax_cross_entropy_with_logits 属于某个分类1，不属于0
    # softmax_cross_entropy_with_logits 每个分类的概率值
    return tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=combine_inputs(X), labels=Y))
    #ValueError: Only call `sparse_softmax_cross_entropy_with_logits` with named arguments (labels=..., logits=..., ...)

def train(total_loss):
    """依据 计算的总损失 训练或调整 模型参数"""
    learning_rate = 0.01
    return tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss)

def evaluate(sess,X,Y):
    """对训练得到的模型进行评估"""
    predicted = tf.cast(tf.argmax(inference(X), 1), tf.int32) 
    print(sess.run(tf.reduce_mean(tf.cast(tf.equal(predicted, Y), tf.float32)))) 

In [6]:
# checkpoints_files="./tmp/checkpoints/ml_softmax"
# saver = tf.train.Saver() #设置检查点

with tf.Session() as sess:
    tf.global_variables_initializer().run()

    X, Y = inputs() 

    total_loss = loss(X, Y)
    train_op = train(total_loss)

    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)

    # actual training loop
    training_steps = 1000
    for step in range(training_steps):
        sess.run([train_op])
        # for debugging and learning purposes, see how the loss gets decremented thru training steps
        if step % 10 == 0:
            print("loss: ", sess.run([total_loss]))

    evaluate(sess, X, Y)
    
    time.sleep(5) #why ?

    coord.request_stop()
    coord.join(threads)
    sess.close()
    

loss:  [1.0911717]
loss:  [1.0083162]
loss:  [0.9705394]
loss:  [0.9471433]
loss:  [0.895168]
loss:  [0.83104044]
loss:  [0.81492156]
loss:  [0.7675319]
loss:  [0.7407849]
loss:  [0.7413286]
loss:  [0.7437897]
loss:  [0.6875938]
loss:  [0.700794]
loss:  [0.66545236]
loss:  [0.63558227]
loss:  [0.6568721]
loss:  [0.6388184]
loss:  [0.5937262]
loss:  [0.60617185]
loss:  [0.6144456]
loss:  [0.55758554]
loss:  [0.6026255]
loss:  [0.5866268]
loss:  [0.54992783]
loss:  [0.5524687]
loss:  [0.5522734]
loss:  [0.53306544]
loss:  [0.54820246]
loss:  [0.52177966]
loss:  [0.51633984]
loss:  [0.5452848]
loss:  [0.47778347]
loss:  [0.4887509]
loss:  [0.5132353]
loss:  [0.50434744]
loss:  [0.48110878]
loss:  [0.5209641]
loss:  [0.4953176]
loss:  [0.4552874]
loss:  [0.51822066]
loss:  [0.45689428]
loss:  [0.44780856]
loss:  [0.5042776]
loss:  [0.46488884]
loss:  [0.43452895]
loss:  [0.45578802]
loss:  [0.48086047]
loss:  [0.4351158]
loss:  [0.48029777]
loss:  [0.4515998]
loss:  [0.39600167]
loss:  [0.